In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import nltk
import PyPDF2
import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords
import requests
import json
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
from summarizer import Summarizer
from flashtext.keyword import KeywordProcessor
from nltk.tokenize import sent_tokenize

In [3]:
nltk.download('stopwords')
nltk.download('popular')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\Hp\AppData\Roaming

True

In [5]:
pdf_file = open(r"C:\Users\Hp\Downloads\Hyderabad.pdf", 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
num_pages = len(pdf_reader.pages)

full_text = ''
for i in range(num_pages):
    page = pdf_reader.pages[i]
    full_text += page.extract_text()

pdf_file.close()


In [6]:
model = Summarizer()
result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)
summarized_text = ''.join(result)
print (summarized_text)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Hyderabad, also known as the "City of Pearls," is a vibrant and bustling city in southern India. With a 
population of over 10 million people, Hyderabad is the capit al city of the Indian state of Telangana. Today, the 
Charminar is one of the most visited tourist attractions in Hyderabad, and it is a symbol of the city's rich 
cultural heritage. The fort has a rich history, having been the site of numerous battles and sieges over 
the centuries. Today, visitors to the fort can explore its sprawling walls and towers, as well as its many 
underground chambers and tunn els. Hyderabad is also known for its delicious cuisine, which is a blend of Mughal, Persian, and local Telugu 
flavors. In addition to its famous landmarks and cuisine, Hyderabad is also a center of technology and 
innovation. The city has taken steps to address these challenges, including investing in public 
transportation and green spaces, and promoting sust ainable development. During the festival, the city comes alive

In [7]:
def get_nouns_multipartite(text):
    out=[]

    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text)
    #    not contain punctuation marks or stopwords as candidates.
    pos = {'PROPN'}
    #pos = {'VERB', 'ADJ', 'NOUN'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')
    extractor.candidate_selection(pos=pos)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1,threshold=0.75,method='average')
    keyphrases = extractor.get_n_best(n=20)

    for key in keyphrases:
        out.append(key[0])

    return out

keywords = get_nouns_multipartite(full_text) 
print (keywords)
filtered_keys=[]
for keyword in keywords:
    if keyword.lower() in summarized_text.lower():
        filtered_keys.append(keyword)
        
print (filtered_keys)

['hyderabad', 'mughals', 'charminar', 'telangana', 'persian', 'hindu', 'ganesha', 'nizams', 'ganesh chaturthi', 'city', 'pearls', '-day', 'bonalu', 'telugu', 'british', 'golconda fort', 'india', 'biryani', 'tunn els', 'orig']
['hyderabad', 'charminar', 'telangana', 'persian', 'city', 'pearls', 'bonalu', 'telugu', 'india', 'tunn els']


In [8]:
def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

In [9]:
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

In [10]:
sentences = tokenize_sentences(summarized_text)
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)       
print (keyword_sentence_mapping)

{'hyderabad': ["Today, the \nCharminar is one of the most visited tourist attractions in Hyderabad, and it is a symbol of the city's rich \ncultural heritage.", 'Another important festival in Hyderabad is the Bonalu, a traditional Telangana festival that celebrates th e \ngoddess Mahankali.', 'Hyderabad is also known for its delicious cuisine, which is a blend of Mughal, Persian, and local Telugu \nflavors.', 'With a \npopulation of over 10 million people, Hyderabad is the capit al city of the Indian state of Telangana.', 'In conclusion, Hyderabad is a city that combines rich history, vibrant culture, and technological \ninnovation.', 'In addition to its famous landmarks and cuisine, Hyderabad is also a center of technology and \ninnovation.', 'Hyderabad, also known as the "City of Pearls," is a vibrant and bustling city in southern India.'], 'charminar': ["Today, the \nCharminar is one of the most visited tourist attractions in Hyderabad, and it is a symbol of the city's rich \ncultur

In [11]:
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

In [12]:
def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

In [13]:
# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

key_distractor_list = {}

In [14]:
for keyword in keyword_sentence_mapping:
    if len(keyword_sentence_mapping[keyword]) > 0:
        wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
        if wordsense:
            distractors = get_distractors_wordnet(wordsense,keyword)
            if len(distractors) == 0:
                distractors = get_distractors_conceptnet(keyword)
            if len(distractors) != 0:
                key_distractor_list[keyword] = distractors
        else:
            distractors = get_distractors_conceptnet(keyword)
            if len(distractors) != 0:
                key_distractor_list[keyword] = distractors

    index = 1

In [15]:
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    print ("%s)"%(index),output)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1

1) Today, the 
Charminar is one of the most visited tourist attractions in  _______ , and it is a symbol of the city's rich 
cultural heritage.
	 a )   Hyderabad
	 b )   Tien Shan
	 c )   Brahmaputra
	 d )   Bangalore

More options:  ['Mumbai', 'Nanda Devi', 'Gujarat', 'Sikkim', 'Tamil Nadu', 'Punjab', 'Assam', 'Tirich Mir', 'Islamabad', 'Rawalpindi', 'Sind', 'Peshawar', 'Great Indian Desert', 'Khyber Pass', 'Kashmir', 'Pamir Mountains'] 


2) Hyderabad is also known for its delicious cuisine, which is a blend of Mughal,  _______ , and local Telugu 
flavors.
	 a )   Afghan
	 b )   Altaic
	 c )   Persian
	 d )   Armenian

More options:  ['Bangladeshi', 'Bengali', 'Bhutanese', 'Burmese', 'Byzantine', 'Cambodian', 'Chinese', 'Coolie', 'East Indian', 'Eurasian', 'Hindu', 'Hmong', 'Iberian', 'Indian', 'Indonesian', 'Irani'] 


3) The  _______  has taken steps to address these challenges, including investing in public 
transportation and green spaces, and promoting sust ainable development.


In [16]:
import tkinter as tk
import re
import random

# Define the main function that generates the quiz
def generate_quiz():
    global sentence_labels, choice_labels
    
    # Clear any existing quiz questions
    for label in sentence_labels:
        label.destroy()
    for label in choice_labels:
        label.destroy()
    
    # Generate a new quiz
    index = 1
    for each in key_distractor_list:
        sentence = keyword_sentence_mapping[each][0]
        pattern = re.compile(each, re.IGNORECASE)
        output = pattern.sub( " _______ ", sentence)
        
        # Create a label for the sentence and add it to the GUI
        sentence_label = tk.Label(root, text="{}. {}".format(index, output))
        sentence_label.pack()
        sentence_labels.append(sentence_label)
        
        choices = [each.capitalize()] + key_distractor_list[each]
        top4choices = choices[:4]
        random.shuffle(top4choices)
        
        # Create labels for the answer choices and add them to the GUI
        for idx,choice in enumerate(top4choices):
            choice_label = tk.Label(root, text="{}. {}".format(optionchoices[idx], choice))
            choice_label.pack()
            choice_labels.append(choice_label)
        
        index = index + 1
    
    # Add a button to generate a new quiz
    generate_button = tk.Button(root, text="Generate Quiz", command=generate_quiz)
    generate_button.pack()

# Define the main window and initialize some variables
root = tk.Tk()
root.title("Keyword Quiz")
sentence_labels = []
choice_labels = []
optionchoices = ['i','ii','iii','iv']

# Call the generate_quiz function to create the initial quiz
generate_quiz()

# Start the main event loop
root.mainloop()
